<a href="https://colab.research.google.com/github/comet-toolkit/comet_training/blob/main/vhroda_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**VHRODA training session: LANDHYPERNET and RADCALNET over Gobabeb**

Within this training session, we will run through processing and comparing a matchup of S2 with HYPERNETS and RadCalNet (both networks providing data for the same overpass) over the Gobabeb site in Namibia. We will assume a starting point where the matchup has already been identified and the files have already been downloaded. The chosen example matchup is on 8th of June 2022
As part of this notebook, we will read in the RadCalNet and HYPERNETS data, mask poor-quality data, and process the data to be comparible with S2 (e.g. spectral band integration). The bias will be calculated together with their uncertainties and plotted. These uncertainties will come from the product files themselves, and will be propagated throughout the processing chain using the CoMet toolkit. The CoMet toolkit will also be used to handle flags and propagate uncertainties for the HYPERNETS products. One example matchup at BOA will be shown, as well as one matchup at TOA. Further matchup can then be processed by the users themselves if time allows. 

We first install the obsarray package (flag handling and accessing uncertainties), the punpy package (uncertainty propagation) and the matheo package (for band integration).

In [ ]:
!pip install obsarray>=1.0.0
!pip install punpy>=1.0.0
!pip install matheo

We also import all the relevant python packages we use in this training:

In [ ]:
import xarray as xr
import numpy as np
from punpy import MeasurementFunction, MCPropagation
from matheo.band_integration import band_integration
from obsarray.templater.dataset_util import DatasetUtil
import matplotlib.pyplot as plt
import os

Next, we clone the comet_training repository, which contains all the datasets used in this training. Some instructions on downloading LANDHYPERNET and RadCalNet data will be provided at the end of this notebook.   

In [ ]:
!git clone https://github.com/comet-toolkit/comet_training.git

**BOA matchup**

In this section, we will compare some BOA LAndsat-9 data to HYPERNETS and RadCalNet. The chosen Landsat-9 product is LC09_L2SP_179076_20220606_20230414_02_T1, which is a tile over Gobabeb from 2022-06-06. Rather than providing the full tile and a reader, we have pre-extracted the regoin of interest (here taken to be a 200 by 200m box centred on the Gobabeb HYPERNETS and GObabeb RadCalNet mast), and make this available as a netcdf file on the comet_training repository. This data can be read in as:

In [ ]:
ds_refl_L9 = xr.load_dataset("example_L9_20220606.nc")     # ROI centred on HYPERNETS
ds_refl_L9_rcn = xr.load_dataset("example_L9_20220606_rcn.nc") # ROI centred on RadCalNet

From these we can determine the reflectance in each band from the mean over the ROI, and its uncertainty (combination of noise and spatial variability) from the standard deviatino between pixels:

In [ ]:
bands_L9=["B1","B2","B3","B4","B5","B6"]
wav_L9 = [442.98244284,482.58889933,561.33224557,654.60554515,864.5708545,1609.09056245]

refl_L9 = np.array([np.mean(ds_refl_L9[band].values) for band in bands_L9])
u_refl_L9 = np.array([np.std(ds_refl_L9[band].values) for band in bands_L9])

refl_L9_rcn = np.array([np.mean(ds_refl_L9_rcn[band].values) for band in bands_L9])
u_refl_L9_rcn = np.array([np.std(ds_refl_L9_rcn[band].values) for band in bands_L9])

In [ ]:
ds_HYP = xr.open_dataset("HYPERNETS_L_GHNA_L2A_REF_20220606T0900_20231226T1435_v2.0.nc")  # read digital effects table

In [ ]:
bad_flags=["pt_ref_invalid", "half_of_scans_masked", "not_enough_dark_scans", "not_enough_rad_scans",
           "not_enough_irr_scans", "no_clear_sky_irradiance", "variable_irradiance",
           "half_of_uncertainties_too_big", "discontinuity_VNIR_SWIR", "single_irradiance_used"]
flagged = DatasetUtil.get_flags_mask_or(ds_HYP["quality_flag"], bad_flags)
id_series_valid = np.where(~flagged)[0]
ds_HYP = ds_HYP.isel(series=id_series_valid)


In [ ]:
vza=0.7
vaa=38.1
vzadiff=(ds_HYP["viewing_zenith_angle"].values - vza)
vaadiff=(np.abs(ds_HYP["viewing_azimuth_angle"].values - vaa%360))
angledif_series = vzadiff** 2 + vaadiff ** 2
id_series = np.where(angledif_series**2 == np.min(angledif_series**2))[0][0]
ds_HYP = ds_HYP.isel(series=id_series)

In [ ]:
wav_HYP_full = ds_HYP["wavelength"].values
refl_HYP_full = ds_HYP["reflectance"].values
u_refl_HYP_full = refl_HYP_full*np.sqrt(ds_HYP["u_rel_systematic_reflectance"].values**2+ds_HYP["u_rel_random_reflectance"].values**2)/100


In [ ]:
class BandIntegrateL9(MeasurementFunction):
    # your measurement function
    def meas_function(self, reflectance, wavelength):
        """
        Function to perform S2A band integration on reflectance

        :param reflectance: reflectance spectrum
        :param wavelength: wavelengths
        """
        refl_band, band_centres = band_integration.spectral_band_int_sensor(
            d=reflectance,
            wl=wavelength,
            platform_name="Landsat-9",
            sensor_name="OLI",
            u_d=None,
        )
        return refl_band[:6]

    def get_argument_names(self):
        """
        Function that returns the argument names of the meas_func, as they appear in the digital effects table (used to find right variable in input data).

        :return: List of argument names
        """
        return ["reflectance", "wavelength"]

    def get_measurand_name_and_unit(self):
        """
        Function that returns the measurand name and unit of the meas_func. These will be used to store in the output dataset.

        :return: tuple(measurand name, measurand unit)
        """
        return "band_reflectance", ""

In [ ]:
prop = MCPropagation(50,parallel_cores=1)
band_int_L8 = BandIntegrateL9(prop, use_err_corr_dict=True)
ds_HYP_L8 = band_int_L8.propagate_ds_total(ds_HYP)


In [ ]:
refl_HYP = ds_HYP_L8["band_reflectance"].values
u_refl_HYP = refl_HYP*ds_HYP_L8["u_tot_band_reflectance"].values/100


In [ ]:

rcn_data = np.genfromtxt("GONA01_2022_157_v00.09.input", dtype="str", delimiter="\t", skip_header=4)
if np.all(rcn_data[:,-1]==''):
    rcn_data=rcn_data[:,:-1]
wav_rcn = rcn_data[12:223, 0].astype(float)
times_rcn = rcn_data[2, 1:]

refl_rcn = rcn_data[12:223, 3].astype(float)
u_refl_rcn = np.abs(rcn_data[229:440, 3].astype(float))

In [ ]:
refl_rcn_band, band_centres, u_refl_rcn_band  = band_integration.spectral_band_int_sensor(
            d=refl_rcn,
            wl=wav_rcn,
            platform_name="Landsat-8",
            sensor_name="OLI",
            u_d=u_refl_rcn,
        )
refl_rcn_band, u_refl_rcn_band = refl_rcn_band[:6], u_refl_rcn_band[:6]

In [ ]:
bias = ((refl_L9/refl_HYP) - 1) * 100
u_bias = (
    np.sqrt(
        (u_refl_L9 / refl_L9) ** 2
        + (u_refl_HYP / refl_HYP) ** 2
    )
    * 100
)

bias_rcn = ((refl_L9_rcn/refl_rcn_band) - 1) * 100
u_bias_rcn = (
    np.sqrt(
        (u_refl_L9_rcn / refl_L9_rcn) ** 2
        + (u_refl_rcn_band / refl_rcn_band) ** 2
    )
    * 100
)


In [ ]:
def plot(path:str, sat:str, name:str, date:str, sat_wav:np.ndarray, sat_refl:np.ndarray, sat_unc:np.ndarray, hyp_wav:np.ndarray, hyp_refl:np.ndarray, hyp_unc:np.ndarray, rcn_wav:np.ndarray,
         rcn_refl:np.ndarray, rcn_unc:np.ndarray, bias:np.ndarray, bias_unc:np.ndarray, wavs_band:np.ndarray, reflectance_band:np.ndarray, reflectance_band_unc:np.ndarray, vza:float, bias_rcn:np.ndarray, bias_rcn_unc:np.ndarray):
    print("preparing %s plot" % name)

    plt.figure(figsize=(20, 12))
    plt.subplot(2, 1, 1)
    plt.errorbar(wavs_band, reflectance_band, yerr=reflectance_band_unc, fmt='o', ls='none', ms=10, color='m',
                 label='HYPERNETS for satellite bands')
    plt.errorbar(sat_wav, sat_refl, yerr=sat_unc, fmt='o', ls='none', ms=10, color='g',
                 label=sat)
    plt.fill_between(hyp_wav, hyp_refl - hyp_unc, hyp_refl + hyp_unc,
                     alpha=0.3, color="b")
    plt.errorbar(rcn_wav, rcn_refl, yerr=rcn_unc,
                 label='RadCalNet', color="orange")
    plt.plot(hyp_wav, hyp_refl, '-b', label='HYPERNETS full-resolution model')
    if sat == "Landsat-8" or sat == "Landsat-9":
        plt.title('Landsat-8/9 (vza=%.1f) vs HYPERNETS TOA Comparison at %s' % (
            vza, date),
                  fontsize=20)
    else:
        plt.title('%s (vza=%.1f) vs HYPERNETS Comparison at %s' % (
            sat, vza, date),
                  fontsize=20)
    plt.ylabel('Reflectance', fontsize=20)
    plt.xlim(380, 1700)
    plt.ylim(0.0, 0.6)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.legend(loc=2, numpoints=1, scatterpoints=1, facecolor='white')

    plt.subplot(2, 1, 2)
    plt.errorbar(sat_wav, bias, yerr=bias_unc, fmt='o', mfc='blue', ls='none', ms=15, capsize=3,
                 label="HYPERNETS-%s bias" % sat)
    plt.errorbar(sat_wav, bias_rcn, yerr=bias_rcn_unc, fmt='o', mfc='orange', ls='none', ms=15, capsize=3, alpha=0.5,
                 label="RadCalNet-%s bias" % sat)
    plt.axhline(y=0, color='r', linestyle='--')
    plt.ylabel('Relative Difference (%)', fontsize=20)
    plt.xlabel('Wavelength (nm)', fontsize=20)
    plt.xlim(380, 1700)
    plt.ylim(-10, 10)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.legend()
    # plt.legend(loc=2, numpoints=1, scatterpoints=1, facecolor='white')
    plt.savefig(os.path.join(path, name + "_comparison.png"), dpi=300, bbox_inches="tight")
    plt.close()


In [ ]:
plot("./", "LANDSAT 9", "bias_L9_test", "2022-06-06", wav_L9, refl_L9, u_refl_L9, wav_HYP_full, refl_HYP_full, u_refl_HYP_full, wav_rcn,
     refl_rcn, u_refl_rcn, bias, u_bias, wav_L9, refl_HYP, u_refl_HYP, vza, bias_rcn, u_bias_rcn)


**TOA matchup**

The TOA RadCalNet can be read in using the same reader as the BOA data. The TOA data is provided in the RadCalNet .output files that can be downloaded from the RadCalNet portal.  SEQ20220608T090043_vs_S2B_MSIL1C_20220608T084559_N0400_R107_T33KWP_20220608T113756.SAFE


In [ ]:

ds_refl_S2 = xr.load_dataset("example_S2_20220628.nc")
ds_refl_S2_rcn = xr.load_dataset("example_S2_20220628_rcn.nc")

HYPERNETS does not (currently) provide TOA reference data. Instead the user would need to propagate the HYPERNETS to TOA themselves using Radiative Transfer (RT) modelling. This was done in De Vis et al (2024a). We here provide the TOA reflectances manually. These were based on propagating the BOA reflectances from the previous section to TOA using LibRadtran. Atmospheric properties from the RadCalNet network were used, and uncertainties from both atmospheric properties as well as on the reflectance were also propagated through the RT forward model. For further details we refer to De Vis et al (2024a).


In [ ]:
bands_S2=["B01","B02","B03","B04","B05","B06","B07","B08","B09","B10","B11","B8A"]
wav_S2 = [442.69504835,  492.43657768,  559.84905824,  664.62175142,
        704.11493669,  740.49182383,  782.75291928,  832.79041366,
        945.05446558, 1373.46188735, 1613.65941706,  864.71079209]
vza_S2=5.6

refl_S2 = np.array([np.mean(ds_refl_S2[band].values) for band in bands_S2])
u_refl_S2 = np.array([np.std(ds_refl_S2[band].values) for band in bands_S2])
refl_S2_rcn = np.array([np.mean(ds_refl_S2_rcn[band].values) for band in bands_S2])
u_refl_S2_rcn = np.array([np.std(ds_refl_S2_rcn[band].values) for band in bands_S2])

Before comparing them to S2, the RadCalNet and LANDHYPERNET data are band integrated with the S2 spectral response functions:

In [ ]:
wav_TOA_HYP_full, refl_TOA_HYP_full, u_refl_TOA_HYP_full = np.load("hypernets_TOA_example_20220608_full.npy")
wav_TOA_HYP_band, refl_TOA_HYP_band, u_refl_TOA_HYP_band = np.load("hypernets_TOA_example_20220608_band.npy")

In [ ]:
rcn_data = np.genfromtxt("GONA01_2022_159_v04.09.output", dtype="str", delimiter="\t", skip_header=4)
if np.all(rcn_data[:,-1]==''):
    rcn_data=rcn_data[:,:-1]
wav_TOA_rcn = rcn_data[15:226, 0].astype(float)
times_rcn = rcn_data[2, 1:]

refl_TOA_rcn = rcn_data[15:226, 3].astype(float)
u_refl_TOA_rcn = np.abs(rcn_data[232:443, 3].astype(float))

In [ ]:
refl_TOA_rcn_band, band_centres, u_refl_TOA_rcn_band = band_integration.spectral_band_int_sensor(
            d=refl_TOA_rcn,
            wl=wav_TOA_rcn,
            platform_name="Sentinel-2B",
            sensor_name="MSI",
            u_d=u_refl_TOA_rcn,
        )
refl_TOA_rcn_band, u_refl_TOA_rcn_band = np.delete(refl_TOA_rcn_band, 11),np.delete(u_refl_TOA_rcn_band,11)

The uncertainty variables in the HYPERNETS products can be accessed simply using xarray, and include error correlation information in the attributes:

In [ ]:
bias_TOA = ((refl_S2/refl_TOA_HYP_band) - 1) * 100
u_bias_TOA = (
    np.sqrt(
        (u_refl_S2 / refl_S2) ** 2
        + (u_refl_TOA_HYP_band / refl_TOA_HYP_band) ** 2
    )
    * 100
)

bias_TOA_rcn = ((refl_S2_rcn/refl_TOA_rcn_band) - 1) * 100
u_bias_TOA_rcn = (
    np.sqrt(
        (u_refl_S2_rcn / refl_S2_rcn) ** 2
        + (u_refl_TOA_rcn_band / refl_TOA_rcn_band) ** 2
    )
    * 100
)

In the output, we see that the err_corr_1_params attribute refers to the error correlation matrix variable. This one is also available in the dataset::

In [ ]:
plot("./", "Sentinel-2", "bias_S2_test", "2022-06-08", wav_S2, refl_S2, u_refl_S2, wav_TOA_HYP_full, refl_TOA_HYP_full, u_refl_TOA_HYP_full, wav_TOA_rcn,
     refl_TOA_rcn, u_refl_TOA_rcn, bias_TOA, u_bias_TOA, wav_TOA_HYP_band, refl_TOA_HYP_band, u_refl_TOA_HYP_band, vza_S2, bias_TOA_rcn, u_bias_TOA

obsarray can be used to conveniently handle uncertainties in the HYPERNETS products.
It can e.g. be used to inspect uncertainty variables for a particular variable, and calculate the total uncertainties:

**Produce further matchups yourself** 